In [ ]:
import json

In [ ]:
file_path = "/content/drive/MyDrive/우동협/공부/논문구현/RNNmodel/DB.json"

In [ ]:
with open(file_path, "r") as file:
  data = json.load(file)
print(type(data))
print(data.keys())

df_train = data['train']
df_val = data['val']
df_test = data['test']
print(type(df_train))

<class 'dict'>
dict_keys(['train', 'val', 'test'])
<class 'list'>


In [ ]:
# 리스트 데이터를 PyTorch Tensor로 변환
def preprocess_data(data):
    X = torch.tensor([[d["return"], d["S&P500_return"]] for d in data[:-1]], dtype=torch.float32)
    y = torch.tensor([d["return"] for d in data[1:]], dtype=torch.float32)
    # 입력 데이터를 torch 형태로 변환!
    # 리스트, 이중 리스트, 다차원 리스트, NumPy 배열을 입력하면 tensor형태로 변환한다.


    return X, y

# Train, Val, Test 데이터 변환
X_train, y_train = preprocess_data(df_train)
X_val, y_val = preprocess_data(df_val)
X_test, y_test = preprocess_data(df_test)

print("Train X:", X_train.shape, "y:", y_train.shape)
print("Val X:", X_val.shape, "y:", y_val.shape)
print("Test X:", X_test.shape, "y:", y_test.shape)

Train X: torch.Size([1665, 2]) y: torch.Size([1665])
Val X: torch.Size([204, 2]) y: torch.Size([204])
Test X: torch.Size([104, 2]) y: torch.Size([104])


### 데이터를 데이터로더 처리

## 개념
### Dataset
- __getitem__()을 이용해 데이터를 한 개씩 불러오기
- __len__()을 이용해 데이터 전체 크기 확인
- RNN 같은 시계열 모델에서는 시퀀스 형태의 데이터도 만들 수 있음
<br>
✅ 하지만 PyTorch의 DataLoader는 Dataset을 상속받은 객체를 필요로 해.<br>
✅ 위의 CustomDataset을 DataLoader에 넣으면 오류가 발생해.

### DataLoader
- 배치 학습(Batch Training) 지원 → 한 번에 여러 개의 데이터를 불러와 학습 속도를 높임.
- shuffle=True → 데이터를 랜덤하게 섞어서 학습 가능 (과적합 방지)
- num_workers → 데이터 로딩을 여러 개의 프로세스로 나눠서 빠르게 처리 가능

In [ ]:
from torch.utils.data import Dataset, DataLoader
# Dataset: 데이터를 정의하고 관리하는 클래스
# DataLoader: Dataset에서 데이터를 배치(batch) 단위로 불러오는 도구


class ReturnDataset(Dataset):
    def __init__(self, X, y, sequence_length=5):
        self.X = X
        self.y = y
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.X) - self.sequence_length

    def __getitem__(self, idx):
        return (
            self.X[idx : idx + self.sequence_length],  # 시퀀스 입력
            self.y[idx + self.sequence_length - 1]  # 타겟 값
        )
    # (sequence_length, input_dim) : 데이터 단위 -> 해당 크기로 데이터를 전달
    # X_sample.shape = (5, 2)  # (시퀀스 길이, 입력 차원)
    # y_sample.shape = (1,)    # (예측할 값)

# Dataset 생성
sequence_length = 5
train_dataset = ReturnDataset(X_train, y_train, sequence_length)
val_dataset = ReturnDataset(X_val, y_val, sequence_length)
test_dataset = ReturnDataset(X_test, y_test, sequence_length)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)  # 검증 데이터는 순서 유지
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # 테스트 데이터도 순서 유지

# DataLoader는 배치 크기로 데이터를 가져온다.
# X_batch.shape = (16, sequence_length, input_dim)  # (배치 크기, 시퀀스 길이, 입력 차원)
# y_batch.shape = (16,)  # (배치 크기)


In [ ]:
import torch
import torch.nn as nn

In [ ]:
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, bias=True, batch_first=True):
        super(CustomLSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias = bias
        self.batch_first = batch_first

        # 가중치 초기화: 입력 → 게이트들 (input, forget, cell, output)
        self.W_ih = nn.ParameterList([nn.Parameter(torch.randn(4 * hidden_size, input_size if i == 0 else hidden_size)) for i in range(num_layers)])
        # 입력 데이터를 4개의 데이터로 나눈다. 신경망을 거치기 때문에 신경망 처리 완료!
        # 4개의 데이터가 향하는 곳 : forget gate, input gate, cell state, output gate

        self.W_hh = nn.ParameterList([nn.Parameter(torch.randn(4 * hidden_size, hidden_size)) for _ in range(num_layers)])
        # 지난 시점의 hidden state 역시 4개의 데이터로 나뉜다.

        # 편향 사용 유무에 따라 bias를 설계
        if bias:
            self.b_ih = nn.ParameterList([nn.Parameter(torch.randn(4 * hidden_size)) for _ in range(num_layers)])
            self.b_hh = nn.ParameterList([nn.Parameter(torch.randn(4 * hidden_size)) for _ in range(num_layers)])
        else:
            self.register_parameter('b_ih', None)
            self.register_parameter('b_hh', None)

    def forward(self, x, states=None):
        if self.batch_first:
            x = x.transpose(0, 1)  # (batch, seq_len, input_size) → (seq_len, batch, input_size)

        seq_len, batch_size, _ = x.size()

        # t=0 일 때, hidden state와 cell state
        if states is None:
            h_t = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
            # (num_layers, batch_size, hidden) 형태의 데이터
            c_t = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        else:
            h_t, c_t = states

        output = []

        for t in range(seq_len):  # 모든 timestep에 대해 계산
            h_new, c_new = [], []
            for layer in range(self.num_layers):
                x_t = x[t] if layer == 0 else h_new[layer - 1]

                gates = (x_t @ self.W_ih[layer].T + self.b_ih[layer] + h_t[layer] @ self.W_hh[layer].T + self.b_hh[layer])

                i, f, g, o = gates.chunk(4, dim=1)  # 4개 게이트로 나누기
                # (batch, hidden * 4)라 dim = 1로 나누면 각 배치 즉 개별 데이터의 형태는 유지되면서 4개의 데이터로 분할 가능

                i, f, o = torch.sigmoid(i), torch.sigmoid(f), torch.sigmoid(o)
                # input, forget, output gate는 sigmoid 함수를 통해 0~1 사이의 가중치 값으로 변경
                g = torch.tanh(g)
                # 입력데이터와 t-1 시점의 hidden state 정보가 더해진 데이터, tanh 함수를 통해 비선형성 추가

                c_new_layer = f * c_t[layer] + i * g
                # 해당 층의 cell state의 정보 중 불필요한 정보는 forget gate를 통해 제거
                # (과거+새로운)정보를 input gate와 더하여 새로운 정보를 얼마나 반영할지 결정


                h_new_layer = o * torch.tanh(c_new_layer)
                # c_new_layer 과거의 정보와 현재의 정보가 더해진 정보
                # output gate를 통해 얼마나 출력할지 결정
                # (batch, hidden)

                h_new.append(h_new_layer)
                # 각 층의 hidden state가 담겨 있다.
                c_new.append(c_new_layer)
                # 각 층의  cell state가 담겨 있다.

            output.append(h_new[-1])
            # 각 시점의 마지막 hiddent state 값을 저장한다.
            h_t, c_t = h_new, c_new
            # 지난 시점의 각 층의 hidden state와 cell state 전달

        output = torch.stack(output)  # (seq_len, batch_size, hidden_size)
        # 기존에는 (batch_size, hidden_size) 값들이 리스트에 시간 순으로 담겨 있었다.
        # 이를 하나의 텐서로 합치는 역할

        if self.batch_first:
            output = output.transpose(0, 1)  # (batch, seq_len, hidden_size)

        return output, (torch.stack(h_t), torch.stack(c_t))  # 최종 hidden/cell state 반환

        # output 각 시점이 마지막 hidden state값
        # (torch.stack(h_t), torch.stack(c_t)) 마지막 시점의 모든 층의 hidden state와 cell state 반환

## 모델 학습

In [ ]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MyLSTM, self).__init__()
        self.lstm = CustomLSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # 최종 예측 레이어

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력 사용
        return out


In [ ]:
# 모델 초기화
model = MyLSTM(input_size=2, hidden_size=10, num_layers=2, output_size=1)

# 손실 함수 및 옵티마이저
criterion = nn.MSELoss()  # 회귀 문제
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # 학습 모드
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()  # (batch, 1) → (batch,)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # 검증 단계
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


Epoch [1/10], Train Loss: 0.3710, Val Loss: 1.7472
Epoch [2/10], Train Loss: 0.1501, Val Loss: 1.3812
Epoch [3/10], Train Loss: 0.1115, Val Loss: 1.2065
Epoch [4/10], Train Loss: 0.0964, Val Loss: 1.1611
Epoch [5/10], Train Loss: 0.0867, Val Loss: 1.1005
Epoch [6/10], Train Loss: 0.0814, Val Loss: 1.0048
Epoch [7/10], Train Loss: 0.0794, Val Loss: 0.9419
Epoch [8/10], Train Loss: 0.0759, Val Loss: 0.9513
Epoch [9/10], Train Loss: 0.0718, Val Loss: 0.8652
Epoch [10/10], Train Loss: 0.0695, Val Loss: 0.7901


In [ ]:
model.eval()
test_loss = 0.0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Final Test Loss: {test_loss:.4f}")


Final Test Loss: 0.0396
